# Week 5: home assignment

## Assignment structure

- DIY: loss scaling (3 points)
- Efficient batching for language modelling (5 points)
- Profiling of the pipeline (2 points)

Your grade for the assignment is the sum of the points for the sections above. Maximum is 10 points.

## DIY: loss scaling (1.5 + 1.5 points)

Let's use a semantic segmentation pipeline for this section. Your task is to train the model in the AMP mode with loss scaler implemented by you. You **can use** `torch.cuda.amp.autocast` and you **cannot use** `torch.cuda.amp.GradScaler()` (you may only for checking your solution).

Let us remind what loss scaling is. Loss scaling is used to avoid the gradient underflow problem, when computing gradients in FP16 precision. The issue here is that while training in full precision, we might acquire rather small values in the gradients, which will vanish when we cast a tensor to a half precision. To fix the problem the following solution is used:

- make a forward pass for the model and compute the loss
- multiply loss value to some factor
- call `.backward()`
- update model's master weights with **unscaled** FP32 gradients

**Note.** Loss scaling might be done in two different ways: static and dynamic ones. In static mode, you choose a factor for scaling only once and use it for the whole training procedure. In dynamic mode you recompute the factor each time you scale the loss. 

For static scaling you will get **1.5 points**, for dynamic scaling you will get additional **1.5 points**. The task is done if you managed to stably achieve high accuracy values (0.985+) within 5 training epochs. For a start, you can run the training in a full precision mode, then try to run in an AMP mode with and without PyTorch loss scaler. You will observe that adding a scaler gives you additional accuracy points.

**Hint.** To make sure that you're doing everything right, you might want to examine gradients' values: (almost) no zeros must be present there.

In [ ]:
# Download and unpack data
# !wget https://www.dropbox.com/s/tc1qo73rrm3gt3m/CARVANA.zip  # Carvana dataset
# !unzip -q CARVANA.zip
# !rm -rf ./train/.DS_Store
# !rm -rf ./train_masks/.DS_Store

In [ ]:
import torch
from torch import nn
from torchvision import transforms
from tqdm.auto import tqdm

from carvana import Carvana
from unet import Unet

In [ ]:
train_dataset = Carvana(
    root=".",
    transform=transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor()
    ])
)

train_loader = torch.utils.data.DataLoader(
    dataset=train_dataset,
    batch_size=128,
    shuffle=True,
    pin_memory=True,
    num_workers=4
)

In [ ]:
def train(train_loader, model, criterion, epoch, num_epochs, device):
    model.train()

    pbar = tqdm(enumerate(train_loader), total=len(train_loader))
    for i, (images, labels) in pbar:
        images = images.to(device)
        labels = labels.to(device)

        with torch.cuda.amp.autocast():
            outputs = model(images)
            loss = criterion(outputs, labels)
        # TODO: your code for loss scaling here
        
        accuracy = ((outputs > 0.5) == labels).float().mean()
            
        pbar.set_description(
            f"Loss: {round(loss.item(), 4)} "
            f"Accuracy: {round(accuracy.item() * 100, 4)}"
        )

In [ ]:
device = "cuda:0"
model = Unet().to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(
    model.parameters(),
    lr=1e-4
)

num_epochs = 5
for epoch in range(0, num_epochs):
    train(train_loader, model, criterion, epoch, num_epochs, device=device)

## Efficient batching for language modelling (1 + 1 + 3 points)

In this part we suggest you examine the efficiency of the three batching approaches we discussed during the seminar. Let us remind you shortly:

**BRAIN**: pad everything to a fixed `max_length`

**BIG BRAIN**: pad only in the `collate_fn`

**ULTRA DUPER BIG BRAIN**: presort data to sample sequences smartly, preserving similar examples length in the batch

___
More formally, we suggest you download [WikiText-103 dataset](https://www.salesforce.com/products/einstein/ai-research/the-wikitext-dependency-language-modeling-dataset/) and implement all of the mentioned approaches. Use the training part for all of the task's sub-problems.

- For naive batching, you will need to implement a Pytorch Dataset class that will parse training data from the source files of the dataset and pad everything to a `max_length=640` of the training samples. For sequences longer than 640 tokens just truncate the overflowing part. **(1 point)**
- For the second approach, you will need to implement the approach from the seminar for this dataset. More specifically, you needed to pad sequences only up to maximum sample length in the current batch. **(1 point)**
- Finally, for the third approach, you will need to make a small trick. While initializing the dataset, you need to split it into the several bins (let's say, python lists) by samples length. For the task we suggest you uniformly split the samples list sorted by sample length. Conduct experiments for 1, 5, 10, 25, 50 bins. While calling a `__getitem__` method, you firstly sample a bin number, then sample the needed examples number form the bin and pad them with collator from the second subtask. **(3 points)**

For each of the implemented methods mock one training epoch and provide min, max, mean and median batch processing times. Use a `pandas.DataFrame` to display the results in the notebook. For mocking a training epoch we suggest you construct a small GPT-2-like model: use `nn.Embedding` layer, `PositionalEncoding` class from `transformer.py` file and a single `nn.TransformerDecoder` layer with hidden size 1024 and 8 heads. For tokenization use `torchtext.data.utils.get_tokenizer("basic_english")`. Run one epoch **without a backward pass**. Make sure you've [warmed up](https://forums.developer.nvidia.com/t/why-warm-up/48565) GPU before computing the statistics and do not forget about asynchronous CUDA kernels execution.

**Note.** In the third subtask you might want to use (not obligatory) a `batch_sampler` in the data loader. For that, you need to inspect the corresponding Pytorch docs [section](https://pytorch.org/docs/stable/data.html#torch.utils.data.Sampler).

## Profiling (2 points)

In this section, you're given a training script for a Transformer model on WikiText2 dataset. Your task is to examine the bottlenecks of the model. You can find the model script in the `transformer.py` file. As you might notice, this is a PyTorch Transformer implementation.

We suppose that in this task you use [PyTorch profiler](https://pytorch.org/tutorials/recipes/recipes/profiler_recipe.html). However, feel free to use any other profiler that we've discussed. In the training function, you can vary the number of steps that are done during one epoch. We suggest you use only one epoch since our goal is not to train a model but to profile its performance.

To complete the task, provide a detailed description of the model performance:
- Forward pass
    - Inspect PositionalEncoding layer
    - Inspect the Embedding layer
    - Inspect Attention layer (both self attention and projections computations)
- Backward pass
    - How long does it take compared to a forward pass?
    
Provide corresponding profiler's outputs and analyse them. We assume that you will analyse all of the mentioned model parts and other parts if you think it is reasonable (their time consumption is comparable).

In [ ]:
# code sourse: https://pytorch.org/tutorials/beginner/transformer_tutorial.html
import math
from typing import Tuple

import torch
from torch import nn, Tensor
import torch.nn.functional as F
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from torch.utils.data import dataset
from torchtext.datasets import WikiText2
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from tqdm.auto import trange

from transformer import generate_square_subsequent_mask, TransformerModel

In [ ]:
train_iter = WikiText2(split="train")
tokenizer = get_tokenizer("basic_english")
vocab = build_vocab_from_iterator(map(tokenizer, train_iter), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

def data_process(raw_text_iter: dataset.IterableDataset) -> Tensor:
    """Converts raw text into a flat Tensor."""
    data = [torch.tensor(vocab(tokenizer(item)), dtype=torch.long) for item in raw_text_iter]
    return torch.cat(tuple(filter(lambda t: t.numel() > 0, data)))

# train_iter was "consumed" by the process of building the vocab,
# so we have to create it again
train_iter, val_iter, test_iter = WikiText2()
train_data = data_process(train_iter)
val_data = data_process(val_iter)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def batchify(data: Tensor, bsz: int) -> Tensor:
    """Divides the data into bsz separate sequences, removing extra elements
    that wouldn't cleanly fit.

    Args:
        data: Tensor, shape [N]
        bsz: int, batch size

    Returns:
        Tensor of shape [N // bsz, bsz]
    """
    seq_len = data.size(0) // bsz
    data = data[:seq_len * bsz]
    data = data.view(bsz, seq_len).t().contiguous()
    return data.to(device)

batch_size = 20
eval_batch_size = 10
train_data = batchify(train_data, batch_size)  # shape [seq_len, batch_size]
val_data = batchify(val_data, eval_batch_size)

In [ ]:
bptt = 35
def get_batch(source: Tensor, i: int) -> Tuple[Tensor, Tensor]:
    """
    Args:
        source: Tensor, shape [full_seq_len, batch_size]
        i: int

    Returns:
        tuple (data, target), where data has shape [seq_len, batch_size] and
        target has shape [seq_len * batch_size]
    """
    seq_len = min(bptt, len(source) - 1 - i)
    data = source[i:i+seq_len]
    target = source[i+1:i+1+seq_len].reshape(-1)
    return data, target

In [ ]:
ntokens = len(vocab)  # size of vocabulary
emsize = 200  # embedding dimension
d_hid = 200  # dimension of the feedforward network model in nn.TransformerEncoder
nlayers = 2  # number of nn.TransformerEncoderLayer in nn.TransformerEncoder
nhead = 2  # number of heads in nn.MultiheadAttention
dropout = 0.2  # dropout probability

model = TransformerModel(ntokens, emsize, nhead, d_hid, nlayers, dropout).to(device)

In [ ]:
import copy
import time

criterion = nn.CrossEntropyLoss()
lr = 5.0  # learning rate
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)

def train(model: nn.Module) -> None:
    model.train()  # turn on train mode
    total_loss = 0.
    log_interval = 200
    start_time = time.time()
    src_mask = generate_square_subsequent_mask(bptt).to(device)

    num_batches = len(train_data) // bptt
    i = 0
    for batch in trange(0, train_data.size(0) - 1, bptt, desc="Epoch progress: "):
        data, targets = get_batch(train_data, i)
        batch_size = data.size(0)
        if batch_size != bptt:  # only on last batch
            src_mask = src_mask[:batch_size, :batch_size]
        output = model(data, src_mask)
        loss = criterion(output.view(-1, ntokens), targets)

        # feel free to comment out this 
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        total_loss += loss.item()
        if batch % log_interval == 0 and batch > 0:
            lr = scheduler.get_last_lr()[0]
            ms_per_batch = (time.time() - start_time) * 1000 / log_interval
            cur_loss = total_loss / log_interval
            ppl = math.exp(cur_loss)
            print(f"| epoch {epoch:3d} | {batch:5d}/{num_batches:5d} batches | "
                  f"lr {lr:02.2f} | ms/batch {ms_per_batch:5.2f} | "
                  f"loss {cur_loss:5.2f} | ppl {ppl:8.2f}")
            total_loss = 0
            start_time = time.time()
        i += 1
            

def evaluate(model: nn.Module, eval_data: Tensor) -> float:
    model.eval()  # turn on evaluation mode
    total_loss = 0.
    src_mask = generate_square_subsequent_mask(bptt).to(device)
    with torch.no_grad():
        for i in range(0, eval_data.size(0) - 1, bptt):
            data, targets = get_batch(eval_data, i)
            batch_size = data.size(0)
            if batch_size != bptt:
                src_mask = src_mask[:batch_size, :batch_size]
            output = model(data, src_mask)
            output_flat = output.view(-1, ntokens)
            total_loss += batch_size * criterion(output_flat, targets).item()
    return total_loss / (len(eval_data) - 1)

In [ ]:
best_val_loss = float("inf")
epochs = 1
best_model = None

for epoch in range(1, epochs + 1):
    epoch_start_time = time.time()
    train(model)
    val_loss = evaluate(model, val_data)
    val_ppl = math.exp(val_loss)
    elapsed = time.time() - epoch_start_time
    print("-" * 89)
    print(f"| end of epoch {epoch:3d} | time: {elapsed:5.2f}s | "
          f"valid loss {val_loss:5.2f} | valid ppl {val_ppl:8.2f}")
    print("-" * 89)

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model = copy.deepcopy(model)

    scheduler.step()